In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [47]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model


In [48]:
data_directory1 = '/kaggle/input/trashnet/dataset-resized'
data_directory2= '/kaggle/input/fresh-and-stale-classification'
data_directory3= '/kaggle/input/trashbox/TrashBox_train_dataset_subfolders/e-waste'


categories = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash', 'e-waste', 'organic']
images = []
labels = []

In [18]:
for category in categories:
    path = os.path.join(data_directory, category)
    label = categories.index(category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        img_array = cv2.imread(img_path)
        img_array = cv2.resize(img_array, (224, 224))  # Resize images to match the input size of ResNet50
        images.append(img_array)
        labels.append(label)


In [19]:
images = np.array(images)  # No need to normalize pixel values for ResNet50
labels = np.array(labels)

In [5]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(6, activation='softmax')  # 6 classes for the 6 types of trash
])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [6]:
# Step 4: Training
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 24s 343ms/step - accuracy: 0.2681 - loss: 1.6738 - val_accuracy: 0.4091 - val_loss: 1.4596
Epoch 2/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 22s 343ms/step - accuracy: 0.4681 - loss: 1.3337 - val_accuracy: 0.4407 - val_loss: 1.3624
Epoch 3/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 41s 337ms/step - accuracy: 0.5017 - loss: 1.2119 - val_accuracy: 0.4980 - val_loss: 1.2317
Epoch 4/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 42s 347ms/step - accuracy: 0.5905 - loss: 1.0602 - val_accuracy: 0.5375 - val_loss: 1.3122
Epoch 5/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 41s 348ms/step - accuracy: 0.6248 - loss: 0.9956 - val_accuracy: 0.5672 - val_loss: 1.2273
Epoch 6/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 22s 339ms/step - accuracy: 0.6661 - loss: 0.8839 - val_accuracy: 0.5968 - val_loss: 1.1516
Epoch 7/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 41s 343ms/step - accuracy: 0.7453 - loss: 0.7199 - val_accuracy: 0.5909 - val_loss: 1.1362
Epoch 8/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 41s 341ms/step - accuracy: 0.7464 - loss: 0.7202 - val_accu

In [7]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test Accuracy:', test_acc)

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.6130 - loss: 1.6036
Test Accuracy: 0.5948616862297058


Pre trained model ResNet-50

In [20]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [21]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(len(categories), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [22]:
for layer in base_model.layers:
    layer.trainable = False


In [23]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 224s 3s/step - accuracy: 0.5085 - loss: 1.2941 - val_accuracy: 0.8004 - val_loss: 0.5972
Epoch 2/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 258s 3s/step - accuracy: 0.8228 - loss: 0.4876 - val_accuracy: 0.8300 - val_loss: 0.4908
Epoch 3/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 214s 3s/step - accuracy: 0.9014 - loss: 0.3183 - val_accuracy: 0.8300 - val_loss: 0.4672
Epoch 4/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 262s 3s/step - accuracy: 0.9170 - loss: 0.2775 - val_accuracy: 0.8439 - val_loss: 0.4653
Epoch 5/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 212s 3s/step - accuracy: 0.9306 - loss: 0.2406 - val_accuracy: 0.8597 - val_loss: 0.4251
Epoch 6/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 262s 3s/step - accuracy: 0.9588 - loss: 0.1807 - val_accuracy: 0.8577 - val_loss: 0.4045
Epoch 7/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 262s 3s/step - accuracy: 0.9632 - loss: 0.1656 - val_accuracy: 0.8656 - val_loss: 0.4142
Epoch 8/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 262s 3s/step - accuracy: 0.9656 - loss: 0.1552 - val_accuracy: 0.8577 - v

In [24]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test Accuracy:', test_acc)

16/16 ━━━━━━━━━━━━━━━━━━━━ 42s 3s/step - accuracy: 0.8652 - loss: 0.3843
Test Accuracy: 0.8675889372825623


In [25]:
from tensorflow.keras.models import load_model


model.save('trained_resnet50_model.h5')


In [ ]:
!pip show flask opencv-python-headless numpy tensorflow scikit-learn pillow


In [27]:
import tensorflow as tf
print(tf.__version__)


2.15.0
